# Clone the repository

In [56]:
%load_ext autoreload
import os 
import sys 
from IPython.display import display, clear_output 
module = "hds"

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [57]:
if not os.path.exists(module):
    !git clone https://github.com/tiagodsilva/$module 
    !mv $module/data . 

In [58]:
# Install packages
cmd = f"pip install -r {module}/requirements.txt"
status = os.system(cmd) 
if status == 0: 
    clear_output() 

In [115]:
%autoreload 
import numpy as np 
import torch 
import yaml 
sys.path.append(module) 
from vihds.config import Config, Trainer 
from vihds.datasets import build_datasets 
from vihds.parameters import Parameters 
from vihds.training import Training 
from vihds.xval import XvalMerge 
from vihds.vae import build_model 

In [116]:
# Users' parameters 
specs = f"{module}/specs/dr_constant_icml.yaml" 
experiment = "EXAMPLE" 
seed = 42             # The RNG seed
epochs = 1            # Training epochs
test_epoch = 20       # Test epochs
plot_epoch = 100      # Plot epochs
train_samples = 20    # Training samples (per datapoint)
test_samples = 20     # Testing samples (per datapoint) 
dreg = True           # DReG estimator 
precision_hidden_layers = None      # Quantity of hidden layers
use_gpu = torch.cuda.is_available() # Use GPU 
folds = 4                           # Cross-validation folds 
# Attributes to evauate the models' accuracy 
heldout = None         # A heldout device (as R33S32_Y81C76) 
split = 1              # Split for cross-validation in heldout 
figures = True         # Whether to draw figures 
verbose = True         # Increase the volume in stdout 

In [117]:
# Display the models' specifications
model_specs = yaml.load(open(specs, "r"), 
                    Loader=yaml.loader.SafeLoader) 
# print(yaml.dump(model_specs)) 

In [118]:
class Namespace(object): 
    def __init__(self, **kwargs): 
        for key, val in kwargs.items(): 
            self.__dict__[key] = val 

In [119]:
args = Namespace( 
    yaml=specs, 
    experiment=experiment,
    seed=seed, 
    epochs=epochs, 
    test_epoch=test_epoch, 
    plot_epoch=plot_epoch, 
    train_samples=train_samples,
    test_samples=test_samples,
    dreg=dreg,
    precision_hidden_layers=precision_hidden_layers,
    gpu=torch.cuda.current_device() if use_gpu else "cpu", 
    folds=folds, 
    heldout=heldout, 
    split=split, 
    figures=figures, 
    verbose=verbose
) 

In [120]:
def run_on_split(args: Namespace, 
                 settings: Namespace, 
                 split: bool=None): 
    """
    Execute a training-testing split. 
    """
    data = build_datasets(args, settings) 
    parameters = Parameters(settings.params) 
    model = build_model(args, settings, data, parameters) 
    training = Training(args, settings, data, parameters, model) 
    # Return the data and the evaluations for the current epoch
    return data, training.run() 

In [121]:
def train(args: Namespace): 
    settings = Config(args) 
    settings.trainer = Trainer(args, add_timestamp=True) 
    data_pair, val_results = run_on_split(args, settings) 
    
    if (val_results is not None) and settings.trainer is not None: 
        xval_merge = XvalMerge(args, settings) 
        xval_merge.add(1, data_pair, val_results) 
        xval_merge.finalize() 
        xval_merge.save() 
        xval_merge.mark_completed(args.experiment) 
        if args.figures: 
            xval_merge.make_writer() 
            xval_merge.make_images() 
            xval_merge.close_writer() 

In [122]:
train(args) 

Processing command-line arguments
- <__main__.Namespace object at 0x7f9e3d8d68e0>
- Setting test_epoch to 1
- Setting plot_epoch to 1
- Setting: np.random.seed(42)
- Setting: torch.manual_seed(42)
- GPU mode computation


hds/vihds/datasets.py:137: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  times_arr = np.asarray(times_list)
hds/vihds/datasets.py:138: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  obs_arr = np.asarray(observations_list)


Initialising encoder
Initialising decoder
---------------------------
Training: split 1 of 4


  0%| | 0/

epoch    1 | train (iwae-elbo = -7252.9907, time = 3.17, total = 3.17)

...

 | val (iwae-elbo = -7554.3672, time = 5.97, total = 5.97)
Preparing cross-validation results
Saving results to results/EXAMPLE_20220813T222914592234
Saving to: results/EXAMPLE_20220813T222914592234
Making summary figure



/home/tiago/anaconda3/envs/viewer/envs/vihds/lib/python3.8/site-packages/numpy/lib/npyio.py:501: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  arr = np.asanyarray(arr)


Making treatment figure


hds/vihds/plotting.py:255: UserWarning: color is redundantly defined by the 'color' keyword argument and the fmt string "kx" (-> color='k'). The keyword argument will take precedence.
  ax.semilogx(


Making species figure
Making global parameters figure
-  ['prec_x', 'prec_rfp', 'prec_yfp', 'prec_cfp', 'e76', 'e81', 'KGR_76', 'KGR_81', 'KGS_76', 'KGS_81', 'KR6', 'KR12', 'KS6', 'KS12', 'nR', 'nS', 'aYFP', 'aCFP', 'dR', 'dS', 'drfp', 'dyfp', 'dcfp', 'a530', 'a480']
Making variable parameters figure
-  ['r', 'K', 'tlag', 'rc']
Making summary device figures
- Pcat-Pcat
- R100-S32
- R100-S34
- R33-S32
- R33-S34
- R33-S175
Making individual device figures
- Pcat-Pcat
- R100-S32
- R100-S34
- R33-S32
- R33-S34
- R33-S175
